# Libs utilizadas

In [44]:
# Bibliotecas padrão do Python
import os
import joblib

# Bibliotecas para manipulação de dados
import pandas as pd

# Machine Learning - modelos
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

# Técnicas para lidar com desbalanceamento
from imblearn.over_sampling import SMOTE

# Carregando os dados
- Os dados foram criados no notebook `02-trata_dados.ipynb`

In [30]:
print("Carregando dados processados:")
X_train = pd.read_parquet('./data/processed/X_train.parquet')
X_test = pd.read_parquet('./data/processed/X_test.parquet')
y_train = pd.read_parquet('./data/processed/y_train.parquet').values.ravel()
y_test = pd.read_parquet('./data/processed/y_test.parquet').values.ravel()

print(f"Dimensões X_train: {X_train.shape}")
print(f"Dimensões X_test: {X_test.shape}")
print(f"Proporção de Attrition no treino: {y_train.mean():.2%}")
print(f"Proporção de Attrition no teste: {y_test.mean():.2%}")

Carregando dados processados:
Dimensões X_train: (747632, 22)
Dimensões X_test: (186909, 22)
Proporção de Attrition no treino: 17.66%
Proporção de Attrition no teste: 17.66%


# Modelagem

In [31]:
# Cria diretório para salvar modelos
os.makedirs("modelos", exist_ok=True)

## Decision tree
- Modelo simples e interpretável que divide os dados com base em regras se-então.

In [32]:
dt_model  = DecisionTreeClassifier(max_depth=20, random_state=42)
dt_model.fit(X_train, y_train)
joblib.dump(dt_model, 'modelos/decision_tree.pkl')

['modelos/decision_tree.pkl']

## Random forest
- Conjunto de várias árvores que reduz overfitting e melhora a generalização.

In [33]:
rf_model  = RandomForestClassifier(
                                    n_estimators=50,
                                    max_depth=10,
                                    n_jobs=-1,
                                    random_state=42)
rf_model.fit(X_train, y_train)
joblib.dump(rf_model, 'modelos/forest.pkl')

['modelos/forest.pkl']

## Regressão Logística
- Modelo linear que estima a probabilidade de classes com base em uma combinação linear das features.

In [34]:
lr_model  = LogisticRegression(max_iter=1000, class_weight='balanced')
lr_model.fit(X_train, y_train)
joblib.dump(lr_model, 'modelos/logistic_regression.pkl')

['modelos/logistic_regression.pkl']

## SVM
- Encontra um hiperplano que separa as classes com a maior margem possível

In [45]:
svm_model = SVC(kernel='linear', probability=True, max_iter=5000, tol=1e-3, random_state=42)
svm_model.fit(X_train, y_train)
joblib.dump(svm_model, 'modelos/svm.pkl')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:246: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


['modelos/svm.pkl']

## XGBoost
- Vombina várias árvores fracas (geralmente árvores de decisão rasas) para formar um modelo forte.

In [36]:
xgb_model  = XGBClassifier( eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)
joblib.dump(xgb_model, 'modelos/xgboost.pkl')

['modelos/xgboost.pkl']

## Rebalanceamento dos dados

In [37]:
# rebalanceamento
smote = SMOTE(random_state=42, n_jobs=-1)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [38]:
# salvando os dados
pd.DataFrame(X_train_resampled, columns=X_train_resampled.columns).to_parquet('./data/processed/X_train_resampled.parquet')
pd.Series(y_train_resampled, name='target').to_frame().to_parquet('./data/processed/y_train_resampled.parquet')

# Retreino dos modelos
- Utilizando dados rebalanceados

## Random Forest

In [39]:
rf_model_r  = RandomForestClassifier(
                                    n_estimators=50,
                                    max_depth=10,
                                    n_jobs=-1,
                                    random_state=42)
rf_model_r.fit(X_train_resampled, y_train_resampled)

joblib.dump(rf_model_r, 'modelos/forest_r.pkl')

['modelos/forest_r.pkl']

## Regressão Logística

In [40]:
lr_model_r  = LogisticRegression(max_iter=1000, class_weight='balanced')
lr_model_r.fit(X_train_resampled, y_train_resampled)

joblib.dump(lr_model_r, 'modelos/logistic_regression_r.pkl')

['modelos/logistic_regression_r.pkl']

## SVM

In [46]:
svm_model_r = SVC(kernel='linear', probability=True, max_iter=5000, tol=1e-3, random_state=42)
svm_model_r.fit(X_train_resampled, y_train_resampled)

joblib.dump(svm_model_r, 'modelos/svm_r.pkl')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:246: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


['modelos/svm_r.pkl']

## XGBoost

In [42]:
xgb_model_r  = XGBClassifier( eval_metric='logloss', random_state=42)
xgb_model_r.fit(X_train_resampled, y_train_resampled)

joblib.dump(xgb_model_r, 'modelos/xgboost_r.pkl')

['modelos/xgboost_r.pkl']